In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 같은 폴더에 있는 데이터 불러오기
event_df = pd.read_csv('./events.csv')
order_df = pd.read_csv('./order_items.csv')

In [3]:
# events 테이블에 product_id 칼럼 추가
event_df['product_id'] = event_df.apply(lambda row : row.uri.split('/')[-1] if row.event_type == 'product' else np.NaN, axis=1)

In [4]:
# 2번 문제 해결을 위해 필요한 데이터 전처리
event_df_sub = event_df[['session_id', 'user_id', 'product_id', 'created_at']]
event_df_sub.isnull().sum()

session_id          0
user_id       1125671
product_id    1586356
created_at          0
dtype: int64

In [5]:
# 2-1 유저 / 상품 쌍을 확인해야 하므로 두 컬럼에만 있는 결측치 제거
event_df_sub = event_df_sub.dropna()
event_df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345607 entries, 14423 to 2430029
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   session_id  345607 non-null  object 
 1   user_id     345607 non-null  float64
 2   product_id  345607 non-null  object 
 3   created_at  345607 non-null  object 
dtypes: float64(1), object(3)
memory usage: 13.2+ MB


In [6]:
# 비교를 위한 key값 데이터 타입 일치화
order_df = order_df.astype({'user_id' : 'string', 'product_id' : 'string'})
event_df_sub = event_df_sub.astype({'user_id' : 'int'})
event_df_sub = event_df_sub.astype({'user_id' : 'string', 'product_id' : 'string'})

In [7]:
# merge를 위한 key값 생성
order_df['check'] = order_df['user_id'] + '/' + order_df['product_id']
event_df_sub['check'] = event_df_sub['user_id'] + '/' + event_df_sub['product_id']

In [8]:
# 결과 확인
merged_df_1 = pd.merge(order_df, event_df_sub, on='check', how='left', suffixes=('_order', '_event'))
merged_df_1.isnull().sum()

id                        0
order_id                  0
user_id_order             0
product_id_order          0
inventory_item_id         0
status                    0
created_at_order          0
shipped_at           120707
delivered_at         224041
returned_at          311003
sale_price                0
check                     0
session_id                0
user_id_event             0
product_id_event          0
created_at_event          0
dtype: int64

In [9]:
# event가 발생하지 않은 유저/상품 쌍 비율
merged_df_1['user_id_event'].isnull().sum() / len(merged_df_1) * 100

0.0

In [10]:
merged_df_1.to_csv('./merged_df_2_1.csv', sep=',', na_rep='NaN')

In [11]:
# 2-2
# order_df와 event_df_sub에서 created_at이 가장 빠른 시간 선택
first_order = order_df.groupby(['user_id', 'product_id'])['created_at'].min().reset_index()
first_event = event_df_sub.groupby(['user_id', 'product_id'])['created_at'].min().reset_index()

# 두 개 데이터프레임 merge하기
merged_df_2 = pd.merge(first_event, first_order, on=['user_id', 'product_id'], suffixes=('_event', '_order'))
merged_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181747 entries, 0 to 181746
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           181747 non-null  string
 1   product_id        181747 non-null  string
 2   created_at_event  181747 non-null  object
 3   created_at_order  181747 non-null  object
dtypes: object(2), string(2)
memory usage: 5.5+ MB


In [12]:
# 구매시각 이전에 event가 발생한 비율
(merged_df_2['created_at_event'] < merged_df_2['created_at_order']).sum() / len(merged_df_2) * 100

100.0

In [13]:
merged_df_2.to_csv('./merged_df_2_2.csv', sep=',', na_rep='NaN')

In [14]:
# 3-1
# 생성일자 칼럼을 날짜 타입으로 변환
event_df['created_at'] = pd.to_datetime(event_df['created_at'], format='mixed')
# session_id별, 생성일자순으로 정렬
event_df = event_df.sort_values(['session_id', 'created_at'])
# 아래의 세션과 같은 세션인지 확인
event_df['session_check'] = event_df['session_id'] == event_df['session_id'].shift(-1)
# 아래의 세션과 시간 차이 계산하여 30분이 넘는지 확인
event_df['time_diff'] = (event_df['created_at'].shift(-1) - event_df['created_at']).dt.total_seconds() / 60 >= 30
# 아래의 세션과 같은 세션이면서 시간 차이가 30분이 넘는지 확인
event_df['check'] = event_df['session_check'] & event_df['time_diff']

In [15]:
# 한 세션에서 30분을 초과한 세션의 비율
sum(event_df['check']) / len(event_df) * 100

3.098607996914427

In [16]:
# 전체 세션중 30분을 초과한 이벤트가 있는 세션의 비율
sum(event_df['check']) / len(event_df['session_id'].unique()) * 100

11.053319428126361

In [17]:
# 3-2
# 30분을 초과한 이벤트 타입
event_df[event_df['check']]['event_type'].unique()

array(['cart', 'product'], dtype=object)

In [18]:
event_df.to_csv('./event_df.csv', sep=',', na_rep='NaN')